###  First unoptimized version packed into one class

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# для TimeSeriesPreprocessorManager
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from prophet import Prophet
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import pmdarima as pm  # INSTALL TO REQ
from scipy import signal
from scipy.stats import zscore, probplot, pearsonr
from joblib import Parallel, delayed
from tbats import TBATS
import holidays
from sklearn.preprocessing import QuantileTransformer
from tbats import TBATS
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler




from prophet.diagnostics import cross_validation, performance_metrics
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

Importing plotly failed. Interactive plots will not work.


# Receiving input

In [3]:
CLICKHOUSE_HOST='localhost'
CLICKHOUSE_PORT=8123
CLICKHOUSE_USER='CLICKHOUSE_USER' # INPUT YOUR CREDENTIAL 
CLICKHOUSE_PASSWORD='CLICKHOUSE_PASSWORD'

In [4]:
# obtain data
import clickhouse_connect
import os
import sys


client = clickhouse_connect.get_client(
                host=CLICKHOUSE_HOST,
                port=CLICKHOUSE_PORT,
                user=CLICKHOUSE_USER,
                password=CLICKHOUSE_PASSWORD
            )

In [6]:
# load time series from db
tablename = "CRYPTO.btcusdt_12h_raw"
ch_response = client.query(f"SELECT * FROM {tablename} ORDER BY Open_time")
ts_data = pd.DataFrame(ch_response.result_rows, columns=ch_response.column_names)

In [7]:
ts_data.head(7)

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ts_id,Source
0,2017-08-17 00:00:00,4261.48,4485.39,4261.32,4427.30,145.708747,2017-08-17 11:59:59,6.356955e+05,582,122.801360,5.367015e+05,btcusdt_12h,binance_api
1,2017-08-17 12:00:00,4436.06,4485.39,4200.74,4285.08,649.441630,2017-08-17 23:59:59,2.819075e+06,2845,493.447181,2.141515e+06,btcusdt_12h,binance_api
2,2017-08-18 00:00:00,4285.08,4371.52,4134.61,4340.31,720.722201,2017-08-18 11:59:59,3.085356e+06,3051,585.408001,2.508148e+06,btcusdt_12h,binance_api
3,2017-08-18 12:00:00,4320.52,4340.31,3938.77,4108.37,479.166063,2017-08-18 23:59:59,2.001602e+06,2182,387.460709,1.620975e+06,btcusdt_12h,binance_api
4,2017-08-19 00:00:00,4108.37,4184.69,3850.00,3957.60,298.518569,2017-08-19 11:59:59,1.214383e+06,1559,213.297785,8.706680e+05,btcusdt_12h,binance_api
5,2017-08-19 12:00:00,3945.12,4149.99,3928.89,4139.98,82.791194,2017-08-19 23:59:59,3.351012e+05,594,61.038257,2.473339e+05,btcusdt_12h,binance_api
6,2017-08-20 00:00:00,4120.98,4211.08,4032.62,4106.53,71.235136,2017-08-20 11:59:59,2.930715e+05,622,46.165527,1.899877e+05,btcusdt_12h,binance_api


In [8]:
ts_data.tail(7)

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ts_id,Source
6147,2026-01-15 12:00:00,96581.07,97165.03,95134.48,95604.80,14054.49528,2026-01-15 23:59:59,1.352538e+09,2932815,7085.80884,6.825182e+08,btcusdt_12h,binance_api
6148,2026-01-16 00:00:00,95604.81,95871.47,95133.03,95429.09,5116.45507,2026-01-16 11:59:59,4.887122e+08,1009516,2187.83828,2.090544e+08,btcusdt_12h,binance_api
6149,2026-01-16 12:00:00,95429.09,95809.08,94293.46,95550.94,6051.25953,2026-01-16 23:59:59,5.753072e+08,1962043,2867.23124,2.726688e+08,btcusdt_12h,binance_api
6150,2026-01-17 00:00:00,95550.94,95578.20,95021.67,95323.86,1694.35238,2026-01-17 11:59:59,1.614328e+08,459123,839.78958,8.001233e+07,btcusdt_12h,binance_api
6151,2026-01-17 12:00:00,95323.85,95639.45,95050.00,95147.77,2586.44899,2026-01-17 23:59:59,2.467292e+08,561404,1220.06325,1.163867e+08,btcusdt_12h,binance_api
6152,2026-01-18 00:00:00,95147.77,95295.39,94876.33,95168.06,2386.33318,2026-01-18 11:59:59,2.269771e+08,513805,1112.37410,1.058134e+08,btcusdt_12h,binance_api
6153,2026-01-18 12:00:00,95168.06,95376.24,94910.00,95183.55,1256.21349,2026-01-18 23:59:59,1.194810e+08,279502,566.66885,5.389214e+07,btcusdt_12h,binance_api


## Development of a Preprocessor for time series

### The first unoptimized version packed into a single class

In [9]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin


class PeriodicityEstimator(BaseEstimator): # removed TransformerMixin t.k transform not needed
    def __init__(self, max_nlags_ratio=0.5, peak_prominence_threshold=0.1,
                 period_threshold=0.8, min_period=2, max_period=None):
        self.max_nlags_ratio = max_nlags_ratio
        self.peak_prominence_threshold = peak_prominence_threshold
        self.period_threshold = period_threshold
        self.min_period = min_period
        self.max_period = max_period
        self.period_ = None # to store the found period

    def fit(self, X, y=None):
          # the fit method does nothing but save the passed parameters
          # Training takes place in GridSearchCV
            return self


    def predict(self, X):
          """Finds the period in data X using object parameters."""

          period = self._find_period(X)
          self.period_=period
          return period


    def _find_period(self, data):
        n = len(data)
        max_nlags = int(self.max_nlags_ratio * n)
        if self.max_period is None:
            max_period_loc = n // 2
        else:
            max_period_loc = self.max_period

        acf_values = acf(data, nlags=max_nlags, fft=True)
        peaks, _ = find_peaks(acf_values, prominence=self.peak_prominence_threshold)

        valid_peaks = peaks[(peaks >= self.min_period) & (peaks <= max_period_loc)]

        if not valid_peaks.size:
            period = 0
        else:
            acf_peak_values = acf_values[valid_peaks]
            strong_peaks = np.where(acf_peak_values >= self.period_threshold)[0]
            if strong_peaks.size > 0:
                period = valid_peaks[strong_peaks[0]]
            else:
                period = valid_peaks[np.argmax(acf_peak_values)]

        return period


class PeriodicityDetector(BaseEstimator):
    def __init__(self, min_period=2, max_period=None, cv=5):
        self.min_period = min_period
        self.max_period = max_period
        self.cv = cv
        self.best_estimator_=None


    def fit(self, X, y=None):
        param_grid = {
            'max_nlags_ratio': [0.1, 0.25, 0.5],
            'peak_prominence_threshold': [0.05, 0.1, 0.2],
            'period_threshold': [0.5, 0.6, 0.7, 0.8],
            'min_period': [self.min_period], # pass min_period and max_period into estimator
            'max_period': [self.max_period]
        }


        estimator = PeriodicityEstimator()
        grid_search = GridSearchCV(estimator, param_grid, scoring=self._score_period, cv=self.cv, n_jobs=-1)
        grid_search.fit(X.to_numpy().reshape(-1, 1), y=[0] * len(X))

        self.best_estimator_ = grid_search.best_estimator_
        return self

    def _score_period(self,y, period_ ,X=None ,**kwargs):
        # period_ is an array of predictions for each Fad, we only need one
        period = period_[0]

        if period == 0:
             return -np.inf

        try:
            from statsmodels.tsa.seasonal import STL
            stl = STL(X, period=period)
            res = stl.fit()
            return -res.resid.var()
        except:
            return -np.inf

    def predict(self,X):
          return self.best_estimator_.predict(X)


# example of using
#index = pd.date_range('1/1/2000', periods=365 * 10, freq='D')
#data = pd.Series(np.sin(2 * np.pi * index.dayofyear / 12) + np.random.randn(len(index)) / 5, index=index)
#detector = PeriodicityDetector(cv=20)
#detector.fit(data)
#period = detector.predict(data)
#print(f"Optimal period: {period}")
#print(f"Best parameters: {detector.best_estimator_.get_params()}")

In [10]:
class TimeSeriesPreprocessorManager:
    """
        Class for time series preprocessing.
        
        Main purpose:
          - Prepare data for machine learning models and neural networks.
          - Perform cleaning, missing value imputation, outlier removal, scaling,
            decomposition, lag feature generation, and additional feature engineering.
        
        Usage example:
            preprocessor = TimeSeriesPreprocessorManager(
                interval='1d', 
                timestamp_column='Open_time', 
                target_column='Open', 
                scaler='robust', 
                lag_features=5
            )
            processed_df = preprocessor.process(raw_df)
    """
    
    def __init__(self, interval, timestamp_column='Open_time', target_column='Open', additional_columns=None, scaler='robust', lag_features=5):
        """
        Initialize the time series preprocessing manager.

        :param interval: Data interval (e.g., '1h', '12h', '1d', '3d', '1w', '1M').
        :param timestamp_column: Name of the timestamp column (e.g., 'Open_time').
        :param target_column: Main column for forecasting (e.g., 'Open').
        :param additional_columns: List of additional columns to process.
        :param scaler: Scaling type ('minmax', 'standard', 'robust', 'quantile', 'log').
                       For financial data, 'robust' is recommended by default.
        :param lag_features: Number of lag features to generate (if specific configurations are not set).
        """
        self.interval = interval
        self.timestamp_column = timestamp_column
        self.target_column = target_column
        self.additional_columns = additional_columns if additional_columns else []
        self.scaler_type = scaler
        self.lag_features = lag_features
        self.original_data = None
        

    def analyze_series(self, data):
        """
            Analyze time series to detect missing values, outliers, and assess stationarity.
            
            Performs:
              - Count of missing values.
              - Outlier detection using Z-score, IQR method, and MAD.
              - Stationarity assessment using ADF test, KPSS test, rolling statistics stability analysis,
                autocorrelation (lag 1), and Q-Q analysis.
            
            :param data: Pandas Series or array of time series values.
            :return: Dictionary with analysis results.
        """
        analysis = {}
    
        # Missing values
        analysis['missing_values'] = data.isnull().sum()
    
        # Outliers: Z-score
        z_scores = zscore(data.dropna())
        analysis['zscore_outliers'] = int((np.abs(z_scores) > 3).sum())
    
        # Outliers: IQR method
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3 - q1
        analysis['iqr_outliers'] = int(((data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr))).sum())
    
        # Outliers: MAD method
        median_val = data.median()
        mad = np.median(np.abs(data - median_val))
        modified_z = 0.6745 * (data - median_val) / mad if mad != 0 else np.zeros_like(data)
        analysis['mad_outliers'] = int((np.abs(modified_z) > 3.5).sum())
    
        # Stationarity: ADF test
        adf_result = adfuller(data.dropna())
        analysis['adf_pvalue'] = adf_result[1]
        stationary_adf = adf_result[1] < 0.05
    
        # Stationarity: KPSS test
        try:
            kpss_result = kpss(data.dropna(), regression='c', nlags="auto")
            analysis['kpss_pvalue'] = kpss_result[1]
            stationary_kpss = kpss_result[1] > 0.05  # Null hypothesis: series is stationary
        except Exception as e:
            analysis['kpss_pvalue'] = np.nan
            stationary_kpss = False
    
        # Rolling statistics stability analysis (mean and std)
        window = max(30, len(data) // 10)  # window size at least 30 observations
        rolling_mean = data.rolling(window=window, min_periods=1).mean()
        rolling_std = data.rolling(window=window, min_periods=1).std()
        mean_cv = rolling_mean.std() / rolling_mean.mean() if rolling_mean.mean() != 0 else np.inf
        std_cv = rolling_std.std() / rolling_std.mean() if rolling_std.mean() != 0 else np.inf
        analysis['rolling_mean_cv'] = mean_cv
        analysis['rolling_std_cv'] = std_cv
    
        # Autocorrelation at lag 1
        autocorr_lag1 = data.autocorr(lag=1)
        analysis['lag1_autocorrelation'] = autocorr_lag1
    
        # Q-Q analysis: calculate correlation between empirical and theoretical quantiles
        qq = probplot(data.dropna(), dist="norm", plot=None)
        analysis['qq_correlation'] = qq[1][2]
    
        # Final stationarity assessment: series is considered stationary if:
        #   - ADF confirms stationarity (p < 0.05),
        #   - KPSS does not reject stationarity hypothesis (p > 0.05),
        #   - Coefficients of variation for rolling mean and std are below threshold.
        rolling_threshold = 0.2
        self._is_stationary = (stationary_adf and stationary_kpss and 
                                  (mean_cv < rolling_threshold) and (std_cv < rolling_threshold))

        analysis['stationary'] = self._is_stationary
    
        return analysis


    def detrend_series(self, data, method='linear'):
        """
        Remove trend from time series.
        
        Two methods:
          - 'linear': Remove linear trend using polynomial regression.
          - 'difference': Differentiate series (remove trend by computing differences).
        
        :param data: Pandas Series with time series.
        :param method: String 'linear' or 'difference'.
        :return: Tuple (detrended_series, trend_component), where detrended_series is the detrended series,
                 and trend_component is the extracted trend component or initial value (for difference).
        """
        if method == 'linear':
            x = np.arange(len(data))
            coeffs = np.polyfit(x, data.values, 1)
            trend = np.polyval(coeffs, x)
            detrended = data - trend
            return detrended, pd.Series(trend, index=data.index)
        elif method == 'difference':
            detrended = data.diff().dropna()
            # For inverse transformation with differencing, save the initial value
            return detrended, data.iloc[0]
        else:
            raise ValueError("Unsupported detrending method. Choose 'linear' or 'difference'.")


    def inverse_detrend(self, detrended_data, trend_component, method='linear'):
        """
        Inverse transformation to restore trend component in time series.
    
        :param detrended_data: Pandas Series without trend.
        :param trend_component: Trend component obtained during detrending.
        :param method: Inverse transformation method ('linear' or 'difference').
        :return: Restored series with trend.
        """
        if method == 'linear':
            return detrended_data + trend_component
        elif method == 'difference':
            # Inverse transformation for differencing: cumulative sum of differences plus initial value
            return detrended_data.cumsum() + trend_component
        else:
            raise ValueError("Unsupported inverse detrending method. Choose 'linear' or 'difference'.")

    
    def fill_missing_values(self, data, method='linear'):
        """
            Fill missing values in data using specified interpolation method.
            
            :param data: Pandas Series of time series.
            :param method: Filling method ('linear', 'mean', etc.).
            :return: Series with filled missing values.
            Example:
                filled_series = preprocessor.fill_missing_values(series, method='linear')
        """
        return data.interpolate(method=method).bfill().ffill()


    def _optimize_contamination(self, data, method='isolation_forest'):
        """
            Optimize contamination parameter (outlier fraction) based on data analysis.
            
            :param data: Pandas Series of time series.
            :param method: Method for outlier removal ('isolation_forest', 'lof', 'auto').
            :return: Optimal contamination value.
        """
        if not isinstance(data, pd.Series):
            data = data[self.target_column]
        
        # Dynamic contamination range determination
        iqr = data.quantile(0.75) - data.quantile(0.25)
        lower_bound = data.quantile(0.25) - 1.5 * iqr
        upper_bound = data.quantile(0.75) + 1.5 * iqr
        estimated_outliers = ((data < lower_bound) | (data > upper_bound)).sum()
        estimated_contamination = max(estimated_outliers / len(data), 0.001)
    
        # Generate logarithmic contamination range
        contamination_range = np.geomspace(max(0.0001, estimated_contamination / 2), 
                                           min(0.51, estimated_contamination * 2), 10)

        best_contamination = 0.001
        best_score = float('inf')
    
        from joblib import Parallel, delayed
    
        def evaluate_contamination(contamination):
            try:
                if method == 'isolation_forest':
                    processed_data = self.remove_outliers_with_isolation_forest(data.copy(), contamination)
                elif method == 'lof':
                    processed_data = self.remove_outliers_with_lof(data.copy(), contamination)
                else:
                    raise ValueError("Unsupported method for contamination optimization.")
    
                # Quality assessment: residual autocorrelation
                residual_diff = data - processed_data
                score = np.mean(np.abs(residual_diff))
    
                # Alternative metric: autocorrelation
                autocorr_score = residual_diff.autocorr()
                final_score = score * (1 - abs(autocorr_score))
    
                return contamination, final_score
            except Exception as e:
                print(f"Error evaluating contamination {contamination}: {e}")
                return contamination, float('inf')
    
        results = Parallel(n_jobs=-1)(delayed(evaluate_contamination)(cont) for cont in contamination_range)
    
        # Select best contamination
        for contamination, score in results:
            if score < best_score:
                best_score = score
                best_contamination = contamination
    
        return best_contamination


    def remove_outliers_with_isolation_forest(self, data, contamination=None):
        """
            Remove outliers using Isolation Forest with automatic contamination tuning.
    
            :param data: Pandas Series of time series.
            :param contamination: Expected fraction of outliers in data. If None, calculated automatically.
            :return: Series with outliers replaced by median.
        """
        if contamination is None:
            # Automatic contamination selection via cross-validation
            contamination = self._optimize_contamination(data, method='isolation_forest')
        reshaped_data = data.values.reshape(-1, 1)
        isol_forest = IsolationForest(contamination=contamination)
        outliers = isol_forest.fit_predict(reshaped_data)
  
        median_value = np.median(data[~np.isin(outliers, -1)])
        data[outliers == -1] = median_value
        return data


    def remove_outliers_with_lof(self, data, contamination=None, n_neighbors=None):
        """
            Remove outliers using Local Outlier Factor (LOF).
    
            :param data: Pandas Series of time series.
            :param contamination: Expected fraction of outliers in data.
            :return: Series with outliers replaced by median.
        """
        if contamination is None:
            # Automatic contamination selection via cross-validation
            contamination = self._optimize_contamination(data, method='lof')

        # Automatic n_neighbors selection if not set
        if n_neighbors is None:
            # Example: choose n_neighbors as 5% of total observations,
            # but not less than 5 and not more than 20
            n_neighbors = min(20, max(5, int(len(data) * 0.05)))

        reshaped_data = data.values.reshape(-1, 1)
        lof = LocalOutlierFactor(contamination=contamination, n_neighbors=n_neighbors)
        outliers = lof.fit_predict(reshaped_data)
        median_value = np.median(data[outliers == 1])
        data[outliers == -1] = median_value
        return data


    def decompose_and_remove_outliers(self, data, method='isolation_forest', contamination=None):
        """
            Combined method: time series decomposition and outlier removal in residuals.
    
            :param data: Pandas Series of time series.
            :param method: Outlier removal method ('isolation_forest', 'lof').
            :param contamination: Expected outlier fraction. If None, calculated automatically.
            :return: DataFrame with updated components (trend, seasonality, residual).
        """
        trend, seasonal, residual = self.decompose_series(data, method='STL')

        if method == 'isolation_forest':
            residual_cleaned = self.remove_outliers_with_isolation_forest(residual, contamination)
        elif method == 'lof':
            residual_cleaned = self.remove_outliers_with_lof(residual, contamination)
        else:
            raise ValueError("Unsupported method for outlier removal.")

        # Reconstruct time series
        cleaned_series = trend + seasonal + residual_cleaned

        return cleaned_series 
        #pd.DataFrame({
        #    f'{self.timestamp_column}': data.index,
        #    f'{self.target_column}': cleaned_series
        #})


    def remove_outliers(self, data, method='combined', contamination=None):
        """
            Select outlier removal method (Isolation Forest, LOF, or combined method).
    
            :param data: Pandas Series of time series.
            :param method: Outlier removal method ('isolation_forest', 'lof', 'combined').
            :param contamination: Expected outlier fraction in data. If None, calculated automatically.
            :return: Series with outliers replaced by median or DataFrame (for combined method).
        """
        if method == 'combined':
            # Automatic method selection
            remove_outliers_method = 'isolation_forest' if len(data) > 1000 else 'lof'
            return self.decompose_and_remove_outliers(data.copy(), method=remove_outliers_method, contamination=contamination)
        elif method == 'isolation_forest':
            return self.remove_outliers_with_isolation_forest(data.copy(), contamination)
        elif method == 'lof':
            return self.remove_outliers_with_lof(data.copy(), contamination)
        else:
            raise ValueError("Unsupported outlier removal method.")


    def determine_period(self, data):
        """
            Determine period (seasonality) of time series using multiple methods.
        
            Uses:
              - PeriodicityDetector,
              - Autocorrelation (ACF),
              - Spectral analysis (FFT).
            
            If period not found, applies heuristics based on self.interval.
            
            :param data: Pandas Series with time series.
            :return: Tuple (period, period_candidates)
        """
        detector = PeriodicityDetector(min_period=2, max_period=None, cv=10)
        detector.fit(data)
        period_detector = detector.predict(data)

        period_candidates = []
        if period_detector > 1:
            print(f"[LOG] Applying PeriodicityDetector!!! {period_detector}")
            period_candidates.append(period_detector)

        # Autocorrelation method (ACF)
        acf_vals = acf(data.dropna(), nlags=min(100, len(data)//2))
        for lag, val in enumerate(acf_vals[1:], start=1):
            if val > 0.3:
                print(f"[LOG] Applying ACF!!! {lag}")
                period_candidates.append(lag)
                break

        # Spectral analysis (FFT)
        fft_data = data.dropna().values
        fft_vals = np.abs(np.fft.rfft(fft_data))
        freqs = np.fft.rfftfreq(len(fft_data), d=1)
        if len(fft_vals) > 1:
            index_candidate = np.argmax(fft_vals[1:]) + 1  # get index excluding zero frequency
            # If obtained index exceeds freqs array bounds, correct it
            if index_candidate >= len(freqs):
                index_candidate = len(freqs) - 1
            dominant_freq = freqs[index_candidate]
            default_max_period = {
                '1h': 168,    # 24*7 - week for hourly data
                '12h': 60,    # approximately 30 days
                '1d': 182,    # approximately half year for daily data
                '3d': 60,     # approximately 180 days/3 ≈ 60 observations for 3-day interval
                '1w': 52,     # 52 weeks (year) for weekly data
                '1M': 12      # 12 months for monthly data
            }
            max_allowed_period = default_max_period.get(self.interval, len(data)/2)
            if dominant_freq > 0:
                period_fft = int(round(1 / dominant_freq))
                if period_fft < max_allowed_period:
                    print(f"[LOG] Applying FFT!!! {period_fft}")
                    period_candidates.append(period_fft)
                else:
                    print(f"[LOG] Ignoring FFT candidate: {period_fft} (exceeds threshold)")

        # If candidates found, take median
        if period_candidates:
            period = int(np.round(np.median(period_candidates)))
        else:
            period = 0  # If nothing found

        
        # Fallback: use heuristic values based on self.interval
        if period < 2:
            print(f"[LOG] Applying Fallback!!!")
            default_periods = {
                '1h': 24,
                '12h': 14,  # option for 12-hour data
                '1d': 7,
                '3d': 7,
                '1w': 52,
                '1M': 12
            }
            period = default_periods.get(self.interval, 1)
        
        print(f"[LOG] Determined period: {period}, candidates: {period_candidates}")
        return period, period_candidates


    def get_seasonality_config(self, interval, data_length):
        """
            Automatic seasonality parameter configuration based on interval and data length.
            Used in decompose_series()/`Prophet`.
            
            :param interval: Data interval (e.g., '1h', '1d', etc.).
            :param data_length: Number of observations.
            :return: Dictionary with seasonality configuration.
        """
        configs = {
            '1h': {'daily': True, 'weekly': False, 'yearly': False},
            '12h': {'daily': True, 'weekly': True, 'yearly': False},
            '1d': {'daily': False, 'weekly': True, 'yearly': True},
            '3d': {'daily': False, 'weekly': True, 'yearly': True},
            '1w': {'daily': False, 'weekly': False, 'yearly': True},
            '1M': {'daily': False, 'weekly': False, 'yearly': True}
        }
        config = configs.get(interval, {'daily': False, 'weekly': False, 'yearly': False})
        if data_length < 365:  # Disable yearly seasonality for short series
            config['yearly'] = False
        return config


    def decompose_series(self, data, method='auto', freq=None, model='auto'):
        """
        Automatic time series decomposition with method selection option.
        """
        if not isinstance(data.index, pd.DatetimeIndex):
            raise ValueError("Data must have a DatetimeIndex for seasonal decomposition.")
    
        adf_pvalue = adfuller(data.dropna())[1]
        model_type = 'additive' if adf_pvalue < 0.05 else 'multiplicative'
        
        period_median, period_candidates = self.determine_period(data)
        method = self._determine_method_auto(data, period_median, period_candidates) if method == 'auto' else method
    
        decomposition_methods = {
            'STL': self._decompose_stl,
            'TBATS': self._decompose_tbats,
            'SSA': self._decompose_ssa,
            'fourier': self._decompose_fourier,
            'prophet': self._decompose_prophet
        }
    
        if method in decomposition_methods:
            return decomposition_methods[method](data, period_median, period_candidates, model_type)
        else:
            raise ValueError(f"Unsupported decomposition method: {method}")


    def _determine_method_auto(self, data, period, candidates):
        if period < 2:
            return 'SSA'
        methods = ['STL', 'SSA', 'fourier']
        quality = {}
        for method in methods:
            result = getattr(self, f'_decompose_{method.lower()}')(data, period, candidates)
            quality[method] = np.nanmean(np.abs(result[2].dropna()))  # Assuming the residual is the third element
        return min(quality, key=quality.get)


    def _decompose_stl(self, data, period, candidates, model_type):
        print('[LOG] STL method selected')
        stl = STL(data, period=period, robust=True) # , seasonal=model_type
        result = stl.fit()
        return result.trend, result.seasonal, result.resid

    
    def _decompose_tbats(self, data, period, candidates, model_type):
        print('[LOG] TBATS method selected')
        from tbats import TBATS
        seasonal_periods = period_candidates if len(period_candidates) > 1 else [period_median]
        if self.interval == '1h':
            seasonal_periods = [24, 168]
        print(f"[LOG] Using seasonal periods for TBATS: {seasonal_periods}")
        estimator = TBATS(seasonal_periods=seasonal_periods)
        tbats_model = estimator.fit(data)
        fitted = tbats_model.y_hat  # fitted values
        try:
            seasonal_array = np.sum(tbats_model.seasonal_components_, axis=1)
        except AttributeError:
            seasonal_array = np.zeros_like(fitted)
        trend_array = fitted - seasonal_array
        resid_array = data.values.flatten() - fitted
        trend = pd.Series(trend_array, index=data.index)
        seasonal = pd.Series(seasonal_array, index=data.index)
        resid = pd.Series(resid_array, index=data.index)
        return trend, seasonal, resid
        

    def _decompose_ssa(self, data, period, candidates, model_type):
        print('[LOG] SSA method selected')
        return self.ssa_decompose(data, window_length=period, variance_threshold=0.9)


    def _decompose_fourier(self, data, period, candidates, model_type):
        print('[LOG] FOURIER method selected')
        return self.fourier_decompose(data, period=period)

    
    def _decompose_prophet(self, data, period, candidates, model_type):
        print('[LOG] Prophet method selected')
        df = pd.DataFrame(data)
        df.reset_index(inplace=True)
        df.columns = ['ds', 'y']
        data_length = len(df)
        seasonality_config = self.get_seasonality_config(self.interval, data_length)
        components = [comp for comp, flag in seasonality_config.items() if flag]
        from fbprophet import Prophet
        model_prophet = Prophet(
            daily_seasonality=seasonality_config.get('daily', False),
            weekly_seasonality=seasonality_config.get('weekly', False),
            yearly_seasonality=seasonality_config.get('yearly', False),
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10,
            seasonality_mode='multiplicative'
        )
        model_prophet.fit(df)
        forecast = model_prophet.predict(df)
        trend_series = pd.Series(forecast['trend'].values, index=data.index)
        seasonal_series = pd.Series(forecast[components].sum(axis=1).values, index=data.index)
        resid_series = pd.Series((df['y'] - forecast['trend'] - forecast[components].sum(axis=1)).values, index=data.index)
        return trend_series, seasonal_series, resid_series


    # DEPRECATED --> TODO: REMOVE 
    def decompose_series_old(self, data, method='auto', freq=None, model='auto'):
        """
        Decompose time series into trend, seasonality, and residual.
        Automatically selects decomposition method (STL, SSA, FOURIER, TBATS, Prophet) based on data analysis.
    
        Parameters:
          :param data   : Pandas Series with DatetimeIndex.
          :param method : Decomposition method ('auto', 'STL', 'TBATS', 'SSA', 'fourier', 'prophet').
                          With 'auto', automatic evaluation between STL, SSA, and FOURIER occurs.
          :param freq: Frequency if specified.
          :param model: Model type ('additive', 'multiplicative', or 'auto').
    
        :return: Tuple (trend, seasonal, residual) – time series components as pandas Series.
        """
        # Index validation
        if not isinstance(data.index, pd.DatetimeIndex):
            raise ValueError("Data must have a DatetimeIndex for seasonal decomposition.")
    
        # Automatic model type selection (additive/multiplicative)
        if model == 'auto':
            adf_pvalue = adfuller(data.dropna())[1]
            model_type = 'additive' if adf_pvalue < 0.05 else 'multiplicative'
            print(f"[LOG] ADF p-value = {adf_pvalue:.4f}, selected model: {model_type}")
        else:
            model_type = model
    
        # Determine period and candidate values
        period_median, period_candidates = self.determine_period(data)
        print(f"[LOG] Determined period (median): {period_median}")
        print(f"[LOG] Period candidates: {period_candidates}")
    
        # If period insufficient, use SSA by default
        if period_median < 2:
            if method == 'auto':
                print("[LOG] Period less than 2, switching to SSA")
                method = 'SSA'
    
        # If method explicitly specified, execute corresponding branch
        if method != 'auto':
            if method.upper() == 'STL' and period_median > 1:
                print('[LOG] STL method explicitly selected')
                stl = STL(data, period=period_median, robust=True)
                result = stl.fit()
                return result.trend, result.seasonal, result.resid
    
            elif method.upper() == 'TBATS':
                print('[LOG] TBATS method explicitly selected')
                from tbats import TBATS
                seasonal_periods = period_candidates if len(period_candidates) > 1 else [period_median]
                if self.interval == '1h':
                    seasonal_periods = [24, 168]
                print(f"[LOG] Using seasonal periods for TBATS: {seasonal_periods}")
                estimator = TBATS(seasonal_periods=seasonal_periods)
                tbats_model = estimator.fit(data)
                fitted = tbats_model.y_hat  # fitted values
                try:
                    seasonal_array = np.sum(tbats_model.seasonal_components_, axis=1)
                except AttributeError:
                    seasonal_array = np.zeros_like(fitted)
                trend_array = fitted - seasonal_array
                resid_array = data.values.flatten() - fitted
                trend = pd.Series(trend_array, index=data.index)
                seasonal = pd.Series(seasonal_array, index=data.index)
                resid = pd.Series(resid_array, index=data.index)
                return trend, seasonal, resid
    
            elif method.upper() == 'SSA':
                print('[LOG] SSA method explicitly selected')
                return self.ssa_decompose(data, window_length=period_median, variance_threshold=0.9)
    
            elif method.lower() == 'fourier':
                print('[LOG] FOURIER method explicitly selected')
                return self.fourier_decompose(data, period=period_median)
    
            else:
                print('[LOG] Prophet method explicitly selected')
                df = pd.DataFrame(data)
                df.reset_index(inplace=True)
                df.columns = ['ds', 'y']
                data_length = len(df)
                seasonality_config = self.get_seasonality_config(self.interval, data_length)
                components = [comp for comp, flag in seasonality_config.items() if flag]
                from fbprophet import Prophet
                model_prophet = Prophet(
                    daily_seasonality=seasonality_config.get('daily', False),
                    weekly_seasonality=seasonality_config.get('weekly', False),
                    yearly_seasonality=seasonality_config.get('yearly', False),
                    changepoint_prior_scale=0.05,
                    seasonality_prior_scale=10,
                    seasonality_mode='multiplicative'
                )
                model_prophet.fit(df)
                forecast = model_prophet.predict(df)
                trend_series = pd.Series(forecast['trend'].values, index=data.index)
                seasonal_series = pd.Series(forecast[components].sum(axis=1).values, index=data.index)
                resid_series = pd.Series((df['y'] - forecast['trend'] - forecast[components].sum(axis=1)).values, index=data.index)
                return trend_series, seasonal_series, resid_series
    
        # Auto mode: evaluate candidates between STL, SSA, and Fourier
        if method == 'auto':
            candidates = {}
            quality = {}  # quality assessed by mean absolute value of residuals (MAE)
            
            # Try STL
            if period_median > 1:
                try:
                    print("[LOG] Trying STL")
                    stl_result = STL(data, period=period_median, robust=True).fit()
                    candidates['STL'] = (stl_result.trend, stl_result.seasonal, stl_result.resid)
                    quality['STL'] = np.nanmean(np.abs(stl_result.resid.dropna()))
                    print(f"[LOG] STL: residual MAE = {quality['STL']:.4f}")
                except Exception as e:
                    print(f"[ERROR] STL error: {e}")
            # Try SSA
            try:
                print("[LOG] Trying SSA")
                ssa_result = self.ssa_decompose(data, window_length=period_median, variance_threshold=0.9)
                candidates['SSA'] = ssa_result
                quality['SSA'] = np.nanmean(np.abs(ssa_result[2].dropna()))
                print(f"[LOG] SSA: residual MAE = {quality['SSA']:.4f}")
            except Exception as e:
                print(f"[ERROR] SSA error: {e}")
            # Try Fourier regression
            try:
                print("[LOG] Trying FOURIER")
                fourier_result = self.fourier_decompose(data, period=period_median)
                candidates['fourier'] = fourier_result
                quality['fourier'] = np.nanmean(np.abs(fourier_result[2].dropna()))
                print(f"[LOG] FOURIER: residual MAE = {quality['fourier']:.4f}")
            except Exception as e:
                print(f"[ERROR] FOURIER error: {e}")
    
            if not quality:
                raise ValueError("Could not perform decomposition with any method in auto mode.")
    
            # Priority: if STL available and its MAE not worse than best candidate (within 10%), choose STL.
            best_quality = min(quality.values())
            if 'STL' in quality and quality['STL'] <= best_quality * 1.1:
                chosen = 'STL'
            else:
                chosen = min(quality, key=quality.get)
    
            print(f"[LOG] Selected method: {chosen} (residual MAE = {quality[chosen]:.4f})")
            return candidates[chosen]
        

    def fourier_decompose(self, data, period):
        """
            Decompose time series using Fourier regression.
            
            :param data: Pandas Series with time series.
            :param period: Determined period.
            :return: Tuple (trend, seasonal, residual) – series components.
        """
        # Convert data to one-dimensional Series
        data_series = pd.Series(data.values.flatten(), index=data.index)
        df = pd.DataFrame({'y': data_series.values}, index=data_series.index)
        df['t'] = np.arange(len(df))
        best_aic = np.inf
        best_model = None
        best_X = None
        max_k = 5  # maximum number of harmonics
        for k in range(1, max_k + 1):
            df_temp = df.copy()
            for i in range(1, k + 1):
                df_temp[f'sin_{i}'] = np.sin(2 * np.pi * i * df_temp['t'] / period)
                df_temp[f'cos_{i}'] = np.cos(2 * np.pi * i * df_temp['t'] / period)
            X = df_temp.drop(columns='y')
            X = sm.add_constant(X)
            model_fit = sm.OLS(df_temp['y'], X).fit()
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_model = model_fit
                best_X = X.copy()
        params = best_model.params
        # 'const' and 't' describe trend, rest - seasonality
        trend_values = best_X[['const', 't']].dot(params[['const', 't']])
        seasonal_values = best_X.drop(columns=['const', 't']).dot(params.drop(labels=['const', 't']))
        trend_series = pd.Series(trend_values, index=df.index)
        seasonal_series = pd.Series(seasonal_values, index=df.index)
        resid_series = data_series - trend_series - seasonal_series
        return trend_series, seasonal_series, resid_series

    
    def ssa_decompose(self, data, window_length=None, variance_threshold=0.9):
        """
            Decompose time series using Singular Spectrum Analysis (SSA).
            
            :param data: Pandas Series with time series.
            :param window_length: Window length for trajectory matrix formation.
            :param variance_threshold: Cumulative variance threshold for component selection.
            :return: Tuple (trend, seasonal, residual) – series components.
        """
        # Convert series to numpy array
        series = data.values.flatten() if isinstance(data, pd.Series) else np.array(data).flatten()
        N = len(series)
        # Determine window length: if window_length not set, use period or N//2
        if window_length is None:
            period = self.determine_period(data)
            window_length = period if period > 1 else N // 2
    
        K = N - window_length + 1
        # Form trajectory (Hankel) matrix
        X = np.empty((window_length, K))
        for i in range(window_length):
            X[i] = series[i:i+K]
        # Singular value decomposition
        U, s, Vt = np.linalg.svd(X, full_matrices=False)
        total_variance = np.sum(s**2)
        explained_variance = np.cumsum(s**2) / total_variance
        # Determine number of components needed to explain given variance threshold
        r = np.searchsorted(explained_variance, variance_threshold) + 1
    
        # Reconstruct matrix using r components
        X_reconstructed = np.zeros_like(X)
        for i in range(r):
            X_reconstructed += s[i] * np.outer(U[:, i], Vt[i, :])
        # Diagonal averaging (reconstruction via Hankelization)
        reconstructed = np.zeros(N)
        counts = np.zeros(N)
        for i in range(window_length):
            for j in range(K):
                reconstructed[i+j] += X_reconstructed[i, j]
                counts[i+j] += 1
        reconstructed /= counts
    
        # Group components to extract trend and seasonality:
        # Hypothesis: first 2 components (if available) represent trend, rest - seasonality.
        if r >= 2:
            X_trend = s[0] * np.outer(U[:, 0], Vt[0, :])
            X_trend += s[1] * np.outer(U[:, 1], Vt[1, :])
            trend_reconstructed = np.zeros(N)
            counts_trend = np.zeros(N)
            for i in range(window_length):
                for j in range(K):
                    trend_reconstructed[i+j] += X_trend[i, j]
                    counts_trend[i+j] += 1
            trend_reconstructed /= counts_trend
            seasonal_reconstructed = reconstructed - trend_reconstructed
        else:
            trend_reconstructed = reconstructed
            seasonal_reconstructed = np.zeros_like(reconstructed)
        residual = series - reconstructed
    
        # Convert results back to Pandas Series
        if hasattr(data, 'index'):
            trend_reconstructed = pd.Series(trend_reconstructed, index=data.index)
            seasonal_reconstructed = pd.Series(seasonal_reconstructed, index=data.index)
            residual = pd.Series(residual, index=data.index)
        
        return trend_reconstructed, seasonal_reconstructed, residual
    

    def scale_data(self, data):
        """
            Scale data using selected algorithm.
            
            Supported algorithms:
              - 'minmax': MinMaxScaler
              - 'standard': StandardScaler
              - 'robust': RobustScaler (robust to outliers)
              - 'quantile': QuantileTransformer (converts distribution to normal)
              - 'log': Logarithmic scaling (np.log1p)
            
            Key statistics logged before and after scaling.
            
            :param data: Pandas Series of time series.
            :return: One-dimensional array of scaled data.
        """
        # Log initial data statistics
        initial_stats = {
            'min': data.min(),
            'max': data.max(),
            'mean': data.mean(),
            'std': data.std(),
            'median': data.median(),
            'IQR': data.quantile(0.75) - data.quantile(0.25)
        }
        print("[LOG] Initial data stats:", initial_stats)
    
        reshaped_data = data.values.reshape(-1, 1)
        
        # Select scaling algorithm based on self.scaler_type parameter
        if self.scaler_type == 'minmax':
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
        elif self.scaler_type == 'standard':
            from sklearn.preprocessing import StandardScaler
            scaler = StandardScaler()
        elif self.scaler_type == 'robust':
            from sklearn.preprocessing import RobustScaler
            scaler = RobustScaler()
        elif self.scaler_type == 'quantile':
            from sklearn.preprocessing import QuantileTransformer
            scaler = QuantileTransformer(output_distribution='normal')
        elif self.scaler_type == 'log':
            # Logarithmic scaling: note that data should be positive
            scaled_data = np.log1p(reshaped_data)
            self.custom_inverse = lambda x: np.expm1(x)
            scaled_data = scaled_data.flatten()
            # Log statistics after scaling
            log_stats = {
                'min': scaled_data.min(),
                'max': scaled_data.max(),
                'mean': scaled_data.mean(),
                'std': scaled_data.std()
            }
            print("[LOG] Data after log scaling stats:", log_stats)
            return scaled_data
        else:
            raise ValueError("Unsupported scaler type. Choose from: 'minmax', 'standard', 'robust', 'quantile', 'log'.")
    
        # Apply selected scaler
        scaled_data = scaler.fit_transform(reshaped_data).flatten()
    
        # Log statistics after scaling
        after_stats = {
            'min': scaled_data.min(),
            'max': scaled_data.max(),
            'mean': scaled_data.mean(),
            'std': scaled_data.std()
        }
        print("[LOG] Scaled data stats:", after_stats)
    
        # Save scaler for inverse transformation
        self.scaler = scaler
        return scaled_data


    def inverse_scale(self, scaled_data):
        """
            Inverse transformation of scaled data to original scale.
            
            Supports inverse transformation for logarithmic scaling via custom_inverse.
            
            :param scaled_data: Scaled data (numpy array).
            :return: Data in original scale (numpy array).
        """
        if self.scaler_type == 'log' and hasattr(self, 'custom_inverse'):
            reshaped_data = scaled_data.reshape(-1, 1)
            inversed = self.custom_inverse(reshaped_data)
            return inversed.flatten()
        else:
            reshaped_data = scaled_data.reshape(-1, 1)
            return self.scaler.inverse_transform(reshaped_data).flatten()

    
    def add_features(self, df, windows=None, holiday_country='US'):
        """
        Add unified feature set to time series DataFrame.
        
        Feature set includes:
          - Lag features (calculated on scaled column).
          - Rolling statistics over multiple windows (mean, standard deviation, min,
            max, median, skewness, kurtosis).
          - Derivative features: first and second derivatives of series.
          - Rolling trend: trend slope calculated via linear regression over window.
          - Financial indicators: returns (pct_change) and volatility (rolling std of returns).
          - Holiday flag: boolean is_holiday feature, True if date is a holiday for specified country.
        
        Dynamic window selection:
          - If windows parameter not passed, window set chosen based on self.interval:
                '1h':  [3, 6, 12, 24]
                '12h': [2, 4, 6, 14]
                '1d':  [3, 7, 14, 30]
                '3d':  [2, 4, 7]
                '1w':  [2, 4, 6]
                '1M':  [3, 6, 12]
          - If self.interval value absent from configuration, default windows [3, 5, 7] used.
        
        Lag configuration similar to previous implementation:
                '1h':  [1, 12, 24, 168]
                '12h': [1, 2, 3, 14]
                '1d':  [1, 7, 14, 30]
                '3d':  [1, 2, 7]
                '1w':  [1, 2, 4]
                '1M':  [1, 3, 6, 12]
          - If self.interval not defined, lags from 1 to self.lag_features used.
        
        Usage example:
             df_features = preprocessor.add_features(raw_df)
        
        :param df: DataFrame with time series, containing at minimum:
                   - self.timestamp_column (e.g., 'Open_time')
                   - self.target_column (e.g., 'Open')
                   also desirable to have scaled data column, which can be named
                   f'{self.target_column}_scaled'. If no such column, original series used.
        :param windows: (optional) List of window sizes for rolling statistics calculation.
        :param holiday_country: Country code for holiday determination (e.g., 'US').
        :return: DataFrame with added features, indexed same as original DataFrame.
        """
        # Create DataFrame copy and convert index to datetime
        df = df.copy()

        if not np.issubdtype(df.index.dtype, np.datetime64):
            df.index = pd.to_datetime(df.index, errors='coerce')
        
        # Determine which column to base feature calculation on.
        # If f'{self.target_column}_scaled' exists, use it, otherwise - original series.
        base_col = f'{self.target_column}_scaled' if f'{self.target_column}_scaled' in df.columns else self.target_column
        
        features = df.copy()
        features[base_col] = df[base_col]
        
        # --- Dynamic window selection ---
        default_windows = {
            '1h': [3, 6, 12, 24],
            '12h': [2, 4, 6, 14],
            '1d': [3, 7, 14, 30],
            '3d': [2, 4, 7],
            '1w': [2, 4, 6],
            '1M': [3, 6, 12]
        }
        if windows is None:
            windows = default_windows.get(self.interval, [3, 5, 7])
        
        # --- Lag features ---
        lag_config = {
            '1h': [1, 12, 24, 168],
            '12h': [1, 2, 3, 14],
            '1d': [1, 7, 14, 30],
            '3d': [1, 2, 7],
            '1w': [1, 2, 4],
            '1M': [1, 3, 6, 12]
        }
        lags = lag_config.get(self.interval, list(range(1, self.lag_features + 1)))
        for lag in lags:
            features[f'lag_{lag}'] = df[base_col].shift(lag)
        
        # --- Rolling statistics (over multiple windows) ---
        # For each given window, calculate set of statistics
        for window in windows:
            # Ensure window not larger than series length
            if window > len(df):
                continue
            features[f'roll_mean_{window}'] = df[base_col].rolling(window=window, min_periods=1).mean()
            features[f'roll_std_{window}'] = df[base_col].rolling(window=window, min_periods=1).std()
            features[f'roll_min_{window}'] = df[base_col].rolling(window=window, min_periods=1).min()
            features[f'roll_max_{window}'] = df[base_col].rolling(window=window, min_periods=1).max()
            features[f'roll_median_{window}'] = df[base_col].rolling(window=window, min_periods=1).median()
            if window > 2:
                features[f'roll_skew_{window}'] = df[base_col].rolling(window=window, min_periods=1).skew()
            if window > 3:
                features[f'roll_kurtosis_{window}'] = df[base_col].rolling(window=window, min_periods=1).kurt() 
        
        # --- Derivative features (based on original target_column) ---
        # Can use original values here, as derivatives often reflect change dynamics
        features['first_derivative'] = df[self.target_column].diff()
        features['second_derivative'] = df[self.target_column].diff(2)
        
        # Rolling trend: calculate trend slope over rolling window for each window from windows
        def calc_trend(x, win):
            if len(x) < win:
                return np.nan
            # Linear regression: return slope
            return np.polyfit(np.arange(len(x)), x, 1)[0]
        
        for window in windows:
            # Apply function using .rolling().apply() method with raw=True
            features[f'rolling_trend_{window}'] = df[self.target_column].rolling(window=window, min_periods=window).apply(lambda x: calc_trend(x, window), raw=True)
        
        # --- Financial indicators ---
        # Returns (pct_change) and volatility (rolling std of returns)
        features['returns'] = df[self.target_column].pct_change()
        # For volatility, can use most suitable window (e.g., median from windows)
        vol_window = windows[len(windows)//2] if windows else 7
        features['volatility'] = features['returns'].rolling(window=vol_window, min_periods=1).std()

        # --- Holidays ---
        try:
            dates_series = pd.Series(df.index, index=df.index)
            years = dates_series.dt.year.unique().tolist()
            country_holidays = holidays.CountryHoliday(holiday_country, years=years, expand=True, observed=True)
            holiday_flags = dates_series.dt.date.map(lambda d: d in list(country_holidays.keys()))
            features['is_holiday'] = pd.Series(holiday_flags.values, index=features.index)
        except Exception as e:
            print(f"[LOG] Failed to load holidays for {holiday_country}: {e}")
            features['is_holiday'] = False
        
        # Remove rows with NaN resulting from shifts and window operations
        features = features.dropna()
        
        return features


    
    def process(self, df):
        """
        Unified time series preprocessing pipeline, returning X and y for ML.
        
        Steps:
          1. Validate and convert timestamps.
          2. Save original data.
          3. Fill missing values for main and additional columns.
          4. Analyze time series (log analysis results).
          5. Remove outliers for main column.
          6. Scale main column.
          7. Generate features using unified method (lag and additional features).
          8. Combine features with timestamp and scaled data.
          9. Form final X matrix (features) and y vector (target variable).
        
        Example:
            X, y = preprocessor.process(raw_df)
        
        :param df: Original DataFrame with time series, containing at minimum columns:
                   self.timestamp_column and self.target_column.
        :return: Tuple (X, y), where X is DataFrame with features, and y is Series with target variable.
        """
        try:
            df = df.copy()

            # Convert timestamps
            if self.timestamp_column not in df.columns:
                raise ValueError(f"Timestamp column '{self.timestamp_column}' missing in DataFrame.")
            if not np.issubdtype(df[self.timestamp_column].dtype, np.datetime64):
                df[self.timestamp_column] = pd.to_datetime(df[self.timestamp_column], errors='coerce')
        
            # Save original data
            self.original_data = df.copy()

            # Set timestamp as index
            df = df.set_index(self.timestamp_column)
            
            # Fill missing values for main and additional columns
            cols_to_process = [self.target_column] + self.additional_columns
            for col in cols_to_process:
                df[col] = self.fill_missing_values(df[col])
        
            # Analyze main series (logging for diagnostics)
            analysis = self.analyze_series(df[self.target_column])
            print("[LOG] Time series analysis:", analysis)
        
            # Remove outliers for main column
            df[f'{self.target_column}_removed_outliers'] = self.remove_outliers(df[self.target_column])
        
            # Scale main column
            scaled_series = self.scale_data(df[f'{self.target_column}_removed_outliers'])
            df[f'{self.target_column}_scaled'] = scaled_series

            # Generate features: lag and additional features
            df = self.add_features(df)

            # Sort by timestamp
            df.sort_index(inplace=True)
            print("[LOG] Pipeline completed. Final observation count:", len(df))

            # Form X and y:
            # y - target variable (scaled value)
            # X - feature matrix, excluding timestamp and target variable
            y = df[f'{self.target_column}_scaled']
            X = df.drop(columns=[f'{self.target_column}_scaled'])

            return X, y
    
        except Exception as e:
            raise Exception(f'[ml_manager/TimeSeriesPreprocessorManager] error: {e}')

In [12]:
ts_data

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ts_id,Source
0,2017-08-17 00:00:00,4261.48,4485.39,4261.32,4427.30,145.708747,2017-08-17 11:59:59,6.356955e+05,582,122.801360,5.367015e+05,btcusdt_12h,binance_api
1,2017-08-17 12:00:00,4436.06,4485.39,4200.74,4285.08,649.441630,2017-08-17 23:59:59,2.819075e+06,2845,493.447181,2.141515e+06,btcusdt_12h,binance_api
2,2017-08-18 00:00:00,4285.08,4371.52,4134.61,4340.31,720.722201,2017-08-18 11:59:59,3.085356e+06,3051,585.408001,2.508148e+06,btcusdt_12h,binance_api
3,2017-08-18 12:00:00,4320.52,4340.31,3938.77,4108.37,479.166063,2017-08-18 23:59:59,2.001602e+06,2182,387.460709,1.620975e+06,btcusdt_12h,binance_api
4,2017-08-19 00:00:00,4108.37,4184.69,3850.00,3957.60,298.518569,2017-08-19 11:59:59,1.214383e+06,1559,213.297785,8.706680e+05,btcusdt_12h,binance_api
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6149,2026-01-16 12:00:00,95429.09,95809.08,94293.46,95550.94,6051.259530,2026-01-16 23:59:59,5.753072e+08,1962043,2867.231240,2.726688e+08,btcusdt_12h,binance_api
6150,2026-01-17 00:00:00,95550.94,95578.20,95021.67,95323.86,1694.352380,2026-01-17 11:59:59,1.614328e+08,459123,839.789580,8.001233e+07,btcusdt_12h,binance_api
6151,2026-01-17 12:00:00,95323.85,95639.45,95050.00,95147.77,2586.448990,2026-01-17 23:59:59,2.467292e+08,561404,1220.063250,1.163867e+08,btcusdt_12h,binance_api
6152,2026-01-18 00:00:00,95147.77,95295.39,94876.33,95168.06,2386.333180,2026-01-18 11:59:59,2.269771e+08,513805,1112.374100,1.058134e+08,btcusdt_12h,binance_api


In [13]:
# Create instance of TimeSeriesPreprocessorManager class
ts_preprocessor_manager = TimeSeriesPreprocessorManager('12h')
X, y = ts_preprocessor_manager.process(ts_data)

/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Time series analysis: {'missing_values': 0, 'zscore_outliers': 0, 'iqr_outliers': 0, 'mad_outliers': 12, 'adf_pvalue': 0.8824802235147571, 'kpss_pvalue': 0.01, 'rolling_mean_cv': 0.8749471392789483, 'rolling_std_cv': 0.7250906003473109, 'lag1_autocorrelation': 0.9995792490385362, 'qq_correlation': 0.9253705686223104, 'stationary': False}
[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36166.72933009934, 'std': 32165.42770707314, 'median': 26438.364999999998, 'IQR': 47448.63407255327}
[LOG] Scaled data stats: {'min': -0.4966942770989614, 'max': 1.983007242234336, 'mean': 0.20502938641444945, 'std': 0.6778448917212686}
[LOG] Pipeline completed. Final observation count: 6140


In [14]:
%timeit ts_preprocessor_manager.process(ts_data)

/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Time series analysis: {'missing_values': 0, 'zscore_outliers': 0, 'iqr_outliers': 0, 'mad_outliers': 12, 'adf_pvalue': 0.8824802235147571, 'kpss_pvalue': 0.01, 'rolling_mean_cv': 0.8749471392789483, 'rolling_std_cv': 0.7250906003473109, 'lag1_autocorrelation': 0.9995792490385362, 'qq_correlation': 0.9253705686223104, 'stationary': False}
[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36161.18329604656, 'std': 32166.638938058924, 'median': 26438.364999999998, 'IQR': 47448.554113987026}
[LOG] Scaled data stats: {'min': -0.49669511410997264, 'max': 1.9830105839255405, 'mean': 0.20491284671581697, 'std': 0.6778715591740715}
[LOG] Pipeline completed. Final observation count: 6140


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Time series analysis: {'missing_values': 0, 'zscore_outliers': 0, 'iqr_outliers': 0, 'mad_outliers': 12, 'adf_pvalue': 0.8824802235147571, 'kpss_pvalue': 0.01, 'rolling_mean_cv': 0.8749471392789483, 'rolling_std_cv': 0.7250906003473109, 'lag1_autocorrelation': 0.9995792490385362, 'qq_correlation': 0.9253705686223104, 'stationary': False}
[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36172.19170485849, 'std': 32170.294907824817, 'median': 26438.364999999998, 'IQR': 47448.68384067977}
[LOG] Scaled data stats: {'min': -0.49669375612468747, 'max': 1.9830051622913896, 'mean': 0.20514429309656151, 'std': 0.6779467506101546}
[LOG] Pipeline completed. Final observation count: 6140


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


...

[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36166.536532327395, 'std': 32167.756621790166, 'median': 26438.364999999998, 'IQR': 47448.63407255327}
[LOG] Scaled data stats: {'min': -0.4966942770989614, 'max': 1.983007242234336, 'mean': 0.20502532311998994, 'std': 0.6778939705943541}
[LOG] Pipeline completed. Final observation count: 6140


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Time series analysis: {'missing_values': 0, 'zscore_outliers': 0, 'iqr_outliers': 0, 'mad_outliers': 12, 'adf_pvalue': 0.8824802235147571, 'kpss_pvalue': 0.01, 'rolling_mean_cv': 0.8749471392789483, 'rolling_std_cv': 0.7250906003473109, 'lag1_autocorrelation': 0.9995792490385362, 'qq_correlation': 0.9253705686223104, 'stationary': False}
[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36125.655142783864, 'std': 32139.228135638587, 'median': 26405.88683208988, 'IQR': 47337.44823866553}
[LOG] Scaled data stats: {'min': -0.4971748099608452, 'max': 1.9883510132054285, 'mean': 0.2053293676010785, 'std': 0.678883589595276}
[LOG] Pipeline completed. Final observation count: 6140
[LOG] Time series analysis: {'missing_values': 0, 'zscore_outliers': 0, 'iqr_outliers': 0, 'mad_outliers': 12, 'adf_

/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_38408/3830646174.py:83: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Applying ACF!!! 1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback!!!
[LOG] Determined period: 14, candidates: [1]
[LOG] STL method selected
[LOG] Initial data stats: {'min': 2870.9, 'max': 120529.35, 'mean': 36172.843801324394, 'std': 32175.48632129042, 'median': 26438.364999999998, 'IQR': 47465.73}
[LOG] Scaled data stats: {'min': -0.4965153806757, 'max': 1.9822930143495108, 'mean': 0.20508435878526254, 'std': 0.6778126455060127}
[LOG] Pipeline completed. Final observation count: 6140
2.19 s ± 635 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Error evaluating contamination 0.51: The 'contamination' parameter of IsolationForest must be a str among {'auto'} or a float in the range (0.0, 0.5]. Got 0.51 instead.
Error evaluating contamination 0.51: The 'contamination' parameter of IsolationForest must be a str among {'auto'} or a float in the range (0.0, 0.5]. Got 0.51 instead.
Error evaluating contamination 0.51: The 'contamination' parameter 